<a href="https://colab.research.google.com/github/BardRimon/Study/blob/main/Introduction_in_NLP/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_5_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80_%D0%9A%D0%BB%D1%8E%D0%BA%D0%B8%D0%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание:
1. Объяснить значение трех UD-тегов синтаксических отношений на примере фрагмента размеченного корпуса.
2. Написать функцию разбиения сложносочиненного предложения из двух частей на простые.
3. Написать функцию нахождения наименьшего общего предка двух токенов в дереве зависимостей.
4. Сравнить три пары предложений двумя методами: сравнением расстояния редактирования деревьев зависимостей (zss) и косинусной мерой между BERT-эмбеддингами.


## задание 1 Ud-теги

In [ ]:
! pip install natasha

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 50.7 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=bfe76d1ca59675f64239d58bcedb533c3fa938b97ffcee69fdc14aa53e066951
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26098 sha256=83c7773c4dd8d16516e9119fcdfb7012b9691416f3ca905f9cba74bfc08e802e
  Stored in directory: /root/.cache/pip/wheels/31/d7/d9/eec6891f78cac19a693bd40ecb8365d2f4613318c145ec9816
Successfully built docopt intervaltree


In [ ]:
from natasha import Segmenter, NewsEmbedding, NewsSyntaxParser, Doc

segmenter = Segmenter()
emb = NewsEmbedding()
syntax_parser = NewsSyntaxParser(emb)

# tokens must be defined for this code to run.
# Based on the output tree, the tokens could be:

text = "Первое издание «Москва-Петушки» быстро разошлось."

# tokens =
# tokens = [
#     "Создатели", "уже", "подписали", "соглашение", "с", "рядом",
#     "игровых", "компаний", ",", "включая", "EA", "и", "Hi-Rez", "."
# ]

doc = Doc(text)
doc.segment(segmenter)
doc.parse_syntax(syntax_parser)
doc.sents[0].syntax.print()

    ┌► Первое         amod
┌►┌─└─ издание        nsubj
│ │ ┌► «              punct
│ └►└─ Москва-Петушки appos
│ └──► »              punct
│   ┌► быстро         advmod
└─┌─└─ разошлось      
  └──► .              punct


## Задание 2: разбиваем сложносочинённое предложение на простые
Написать функцию разбиения сложносочиненного предложения из двух частей на простые.

В русском языке зачастую это сочетание сс - союза и punct пунктуации

In [ ]:
!pip install -U spacy
!python -m spacy download ru_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 22.8 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.6
    Uninstalling spacy-3.8.6:
      Successfully uninstalled spacy-3.8.6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 64.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
nlp = spacy.load("ru_core_news_sm")

def split_compound_sentence(sentence):

    doc = nlp(sentence)
    simple_sentences = []
    split_index = -1


    for i, token in enumerate(doc):
        if token.dep_ == "cc" or token.dep_ == 'mark':
            split_index = i
            break

    if split_index != -1:

        sent1_tokens = doc[:split_index - 1]
        sentence1 = "".join(token.text_with_ws for token in sent1_tokens).strip()

        sent2_tokens = doc[split_index + 1:]
        sentence2 = "".join(token.text_with_ws for token in sent2_tokens).strip()
        # починить заглавную букву
        sentence1 = sentence1[0].upper() + sentence1[1:]
        sentence2 = sentence2[0].upper() + sentence2[1:]


        if not sentence1.endswith(('.', '!', '?')):
             sentence1 += '.'
        if not sentence2.endswith(('.', '!', '?')):
             sentence2 += '.'


        simple_sentences.append(sentence1)
        simple_sentences.append(sentence2)
    else:
        # Если союз не найден, возвращаем исходное предложение как одно целое
        simple_sentences.append(sentence)

    return simple_sentences

In [ ]:
sentence = "Было тихо, и над деревней стоял теплый туман."


print(f"\nИсходное предложение: '{sentence}'")
split_sentences = split_compound_sentence(sentence)
print(f"Разбитые предложения: {split_sentences}")


Исходное предложение: 'Было тихо, и над деревней стоял теплый туман.'
Разбитые предложения: ['Было тихо.', 'Над деревней стоял теплый туман.']


## Задание 3: функция нахождения наименьшего предка
Написать функцию нахождения наименьшего общего предка двух токенов в дереве зависимостей.



In [ ]:
text = "Прислонясь к дверному косяку я ловлю в далёком отголоске, что случиться на моём веку"
doc = nlp(text)
# print(token1.dep_, token1.head)
spacy.displacy.render(doc, style="dep", options={"distance": 100})


def get_ancestors(token):

    path = []
    current = token
    while current:
        path.append(current)
        if current.head == current:
            break
        current = current.head
    return path

def lca(token1, token2):

    ancestors1 = get_ancestors(token1)
    ancestors2 = get_ancestors(token2)
    for t1 in reversed(ancestors1):
        if t1 in ancestors2:
            return t1
    return None

# Пример использования
token1 = doc[4]
token2 = doc[8]

common_ancestor = lca(token1, token2)
print(f"Наименьший общий предок: {common_ancestor.text} ({common_ancestor.pos_}) для {token1} и {token2}")



Наименьший общий предок: ловлю (VERB) для я и отголоске


## Задание 4:сравнить сравнение zss и cos
Сравнить три пары предложений двумя методами: сравнением расстояния редактирования деревьев зависимостей (zss) и косинусной мерой между BERT-эмбеддингами.

In [ ]:
! pip install zss torch transformers scikit-learn

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.1 MB/s eta 0:00:00
  Created wheel for zss: filename=zss-1.2.0-py3-none-any.whl size=6725 sha256=77b86e58cec690851fb60c3a6f58a66da0237c4e3be0765f9eb57627c8309cda
  Stored in directo

In [ ]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFBertModel.from_pretrained("bert-base-multilingual-cased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
import torch
# from transformers import BertTokenizer, BertModel
# from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from zss import simple_distance, Node

def build_zss_tree(token):
    node = Node(token.text)
    for child in token.children:
        node.children.append(build_zss_tree(child))
    return node

def compare_sentences_zss(sentence1, sentence2, nlp):
    doc1 = nlp(sentence1)
    doc2 = nlp(sentence2)

    if not doc1.has_annotation("DEP") or not doc2.has_annotation("DEP"):
        return None # Or handle appropriately

    # Find the root of the dependency tree
    root1 = None
    for token in doc1:
        if token.dep_ == "ROOT":
            root1 = token
            break

    root2 = None
    for token in doc2:
        if token.dep_ == "ROOT":
            root2 = token
            break

    if root1 is None or root2 is None:
        return None # Or handle sentences without a clear root

    tree1 = build_zss_tree(root1)
    tree2 = build_zss_tree(root2)

    return simple_distance(tree1, tree2)

def compare_sentences_bert(sentence1, sentence2, model, tokenizer):
    # Change return_tensors to 'tf' to generate TensorFlow tensors
    inputs1 = tokenizer(sentence1, return_tensors='tf', padding=True, truncation=True, max_length=512)
    inputs2 = tokenizer(sentence2, return_tensors='tf', padding=True, truncation=True, max_length=512)


    embeddings1 = model(**inputs1).last_hidden_state
    embeddings2 = model(**inputs2).last_hidden_state

    # Use TensorFlow operations for mean and cosine similarity
    import tensorflow as tf
    embeddings1_mean = tf.reduce_mean(embeddings1, axis=1)
    embeddings2_mean = tf.reduce_mean(embeddings2, axis=1)

    similarity = cosine_similarity(embeddings1_mean, embeddings2_mean)
    return similarity[0][0]

In [ ]:
sentence_pairs = [
    ("Никто этого не знает, и никогда теперь не узнает.", "Площадь Курского вокзала."),
    ("Было тихо, и над деревней стоял теплый туман.", "Стоял теплый туман, и было тихо."),
    ("Очи чёрные, очи страстные!", "Очи жгучие и прекрасные!")
]

# Perform comparisons
for s1, s2 in sentence_pairs:
    print(f"\nСравнение предложений:")
    print(f"Предложение 1: '{s1}'")
    print(f"Предложение 2: '{s2}'")

    # ZSS comparison
    zss_distance = compare_sentences_zss(s1, s2, nlp)
    if zss_distance is not None:
        print(f"Расстояние редактирования деревьев зависимостей (ZSS): {zss_distance}")
    else:
        print("Не удалось построить деревьев зависимостей для ZSS сравнения.")


    # BERT comparison
    bert_similarity = compare_sentences_bert(s1, s2, model, tokenizer)
    print(f"Косинусная мера между BERT-эмбеддингами: {bert_similarity:.4f}")


Сравнение предложений:
Предложение 1: 'Никто этого не знает, и никогда теперь не узнает.'
Предложение 2: 'Площадь Курского вокзала.'
Расстояние редактирования деревьев зависимостей (ZSS): 40.0
Косинусная мера между BERT-эмбеддингами: 0.4121

Сравнение предложений:
Предложение 1: 'Было тихо, и над деревней стоял теплый туман.'
Предложение 2: 'Стоял теплый туман, и было тихо.'
Расстояние редактирования деревьев зависимостей (ZSS): 34.0
Косинусная мера между BERT-эмбеддингами: 0.8841

Сравнение предложений:
Предложение 1: 'Очи чёрные, очи страстные!'
Предложение 2: 'Очи жгучие и прекрасные!'
Расстояние редактирования деревьев зависимостей (ZSS): 21.0
Косинусная мера между BERT-эмбеддингами: 0.9088
